In [11]:
!pip install datasets>=2.6.1
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio

  Cloning https://github.com/huggingface/transformers to c:\users\hp\appdata\local\temp\pip-req-build-tc_mf3gw
  Resolved https://github.com/huggingface/transformers to commit 4b236aed7618d90546cd2e8797dab5b4a24c5dce
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/269.7 kB ? eta -:--:--
   ---------------------------------------- 0.0/269.7 kB ? eta -:--:--
   - -------------------------------------- 10.2/269.7 kB ? eta -:--:--
   -------- ------------------------------ 61.4/269.7 kB 648.1 kB/s eta 0:00:01
   ------------------------------------ --- 245.8/269.7 kB 1.9 MB/s eta 0:00:01
   ---------------------------------------- 269.7/269.7 kB 1.7 MB/s eta 0:00:00

  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers 'C:\Users\HP\AppData\Local\Temp\pip-req-build-tc_mf3gw'
  You can safely remove it manually.
  You can safely remove it manually.


   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   - -------------------------------------- 0.1/1.6 MB 1.7 MB/s eta 0:00:01
   ------ --------------------------------- 0.2/1.6 MB 3.0 MB/s eta 0:00:01
   -------------- ------------------------- 0.6/1.6 MB 4.2 MB/s eta 0:00:01
   ------------------------ --------------- 1.0/1.6 MB 5.8 MB/s eta 0:00:01
   ------------------------------------- -- 1.5/1.6 MB 7.0 MB/s eta 0:00:01
   ---------------------------------------- 1.6/1.6 MB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached importlib_resources-6.1.1-py3-none-any.whl.metadata (4.1 kB)
     ---------------------------------------- 0.0/50.6 kB ? eta -:--:--
     ---------------------------------------- 50.6/50.6 kB 1.3 MB/s eta 0:00:00
  Using cached typer-0.9.0-py3-none-any.whl (45 kB)
     ---------------------------------------- 0.0/124.7 kB ? eta -:--:--
     ---------------

In [33]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset
import numpy as np
import librosa
import librosa.display
from IPython.display import Audio

device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

In [34]:
model_id = "openai/whisper-base"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)


config.json:   0%|          | 0.00/1.98k [00:00<?, ?B/s]

C:\Users\HP\anaconda3\envs\LLM\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\HP\.cache\huggingface\hub\models--openai--whisper-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/290M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.81k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.08k [00:00<?, ?B/s]

In [35]:
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)


In [36]:



pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)



In [37]:
dataset = load_dataset("distil-whisper/librispeech_long", "clean", split="validation")
sample = dataset[0]["audio"]


In [38]:
sample

{'path': '0d38672e0bbdbdc460af55b8bb84a15b2730db2819f2af64f9c777d4d586f2de',
 'array': array([0.00238037, 0.0020752 , 0.00198364, ..., 0.00024414, 0.00048828,
        0.0005188 ]),
 'sampling_rate': 16000}

In [39]:
result = pipe(sample, generate_kwargs={"language": "english"})

print(result["text"])

 Mr. Quilter is the apostle of the middle classes, and we are glad to welcome his gospel. Nor is Mr. Quilter's manner less interesting than his matter. He tells us that at this festive season of the year, with Christmas and roast beef looming before us, similarly he's drawn from eating and its results occur most readily to the mind. He has graved doubts whether Sir Frederick Layton's work is really Greek after all, and can discover in it but little of Rocky Ithaca. Lynelle's pictures are a sort of upgards and atom paintings, and Mason's exquisite ittles are as national as a jingo poem. Mr. Birk at Foster's landscapes smile at one much in the same way that Mr. Carker used to flash his teeth. And Mr. John Collier gives his sitter a cheerful slap on the back before he says like a shampoo in a Turkish bath. Next man,


In [47]:
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()

common_voice["train"] = load_dataset("lj_speech",split='train[:10%]', use_auth_token=True)
#common_voice["validation"] = load_dataset("lj_speech",  split="validation", use_auth_token=True)

print(common_voice)

Using the latest cached version of the dataset since lj_speech couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'main' at C:\Users\HP\.cache\huggingface\datasets\lj_speech\main\1.1.0\af8fa32e4f43a1251b5a2d9cc121181d66575939 (last modified on Wed Feb  7 14:45:53 2024).


DatasetDict({
    train: Dataset({
        features: ['id', 'audio', 'file', 'text', 'normalized_text'],
        num_rows: 1310
    })
})


In [60]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

In [49]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", task="transcribe")

tokenizer_config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.08k [00:00<?, ?B/s]

In [50]:
from transformers import WhisperProcessor, Trainer, TrainingArguments

processor = WhisperProcessor.from_pretrained("openai/whisper-small", task="transcribe")

In [51]:
print(common_voice["train"][0])

{'id': 'LJ001-0001', 'audio': {'path': 'LJ001-0001.wav', 'array': array([-7.32421875e-04, -7.62939453e-04, -6.40869141e-04, ...,
        7.32421875e-04,  2.13623047e-04,  6.10351562e-05]), 'sampling_rate': 22050}, 'file': '/storage/hf-datasets-cache/heavy/datasets/99585803243503-config-parquet-and-info-lj_speech-4e2d0af0/downloads/extracted/034012ae2b5e0aa215c21354dfc93ab72d895d00717e088a2affe73ee49d0b98/LJSpeech-1.1/wavs/LJ001-0001.wav', 'text': 'Printing, in the only sense with which we are at present concerned, differs from most if not from all the arts and crafts represented in the Exhibition', 'normalized_text': 'Printing, in the only sense with which we are at present concerned, differs from most if not from all the arts and crafts represented in the Exhibition'}


In [52]:
from datasets import Audio

common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

In [53]:
print(common_voice["train"][0])

{'id': 'LJ001-0001', 'audio': {'path': 'LJ001-0001.wav', 'array': array([-0.00061751, -0.00074496, -0.00068972, ...,  0.00068615,
        0.00013802,  0.        ]), 'sampling_rate': 16000}, 'file': '/storage/hf-datasets-cache/heavy/datasets/99585803243503-config-parquet-and-info-lj_speech-4e2d0af0/downloads/extracted/034012ae2b5e0aa215c21354dfc93ab72d895d00717e088a2affe73ee49d0b98/LJSpeech-1.1/wavs/LJ001-0001.wav', 'text': 'Printing, in the only sense with which we are at present concerned, differs from most if not from all the arts and crafts represented in the Exhibition', 'normalized_text': 'Printing, in the only sense with which we are at present concerned, differs from most if not from all the arts and crafts represented in the Exhibition'}


In [64]:
from datasets import load_metric
import torch

metric = load_metric("wer")

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # Replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # We do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}



C:\Users\HP\AppData\Local\Temp\ipykernel_1516\601695158.py:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("wer")
C:\Users\HP\anaconda3\envs\LLM\lib\site-packages\datasets\load.py:752: FutureWarning: The repository for wer contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/wer/wer.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [66]:
from transformers import WhisperProcessor, Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=8,
    logging_dir="./logs",
    logging_steps=1000,
    save_steps=1000,
    evaluation_strategy="steps",
    eval_steps=1000,
    save_total_limit=2,
)


RuntimeError: Failed to import transformers.trainer because of the following error (look up to see its traceback):
cannot import name 'is_torch_greater_or_equal_than_1_13' from 'transformers.pytorch_utils' (C:\Users\HP\anaconda3\envs\LLM\lib\site-packages\transformers\pytorch_utils.py)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    #eval_dataset=dataset["validation"],
    compute_metrics=compute_metrics,
)

trainer.train()